In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from collections import defaultdict
import json

In [9]:
for year in range(17, 19):
    print(year)
    file_path = './data/race_data/20' + str(year) + '.csv'
    df = pd.read_csv(file_path)
    df['S13'] = df['S12']
    
    new_times_lap = []
    for index, time in df['Lap Time'].iteritems():
        if time.count(':') == 1:
            new_times_lap.append(datetime.strptime(time, '%M:%S.%f'))
        else:
            new_times_lap.append(datetime.strptime(time, '%H:%M:%S.%f'))
    df['Lap Time'] = new_times_lap
    
    new_times_s = []
    new_day = False
    for index, time in df['Session Time'].iteritems():
        if time.count(':') == 0:
            new_day = True
            if float(time) < 10: 
                new_time = '01:00:00:0' + time
            else:
                new_time = '01:00:00:' + time
            new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
            new_date_plus = new_date + timedelta(days=1)
            new_times_s.append(new_date_plus)
        
        elif time.count(':') == 1:
            new_date_plus = datetime.strptime(time, '%M:%S.%f')
            if new_day: 
                new_date_plus = new_date_plus + timedelta(days=1)
                
            new_times_s.append(new_date_plus)
            
        else:
            new_times_s.append(datetime.strptime(time, '%H:%M:%S.%f'))
            
    df['Session Time'] = new_times_s
    
    df['Prev Session Time'] = df['Session Time'].subtract(df['Lap Time'])
    
    for i in range(1,14):
        num_str = str(i)
        if i < 10: num_str = '0' + num_str
        num_str = 'S' + num_str

        vals = []
        for ind, split_time in df[num_str].iteritems():
            vals.append(timedelta(seconds=split_time))
        df[num_str] = vals
        totals = df[num_str] + df['Prev Session Time']

        for j in range(i-1, 0, -1):
            new_num_str = str(j)
            if j < 10: new_num_str = '0' + new_num_str
            new_num_str = 'S' + new_num_str
            totals += df[new_num_str]
        df[num_str+'totals'] = totals
        
    df.to_csv('./data/race_data/20' + str(year) + '_mod.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

17
18
